In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

from xgboost import XGBClassifier
import os

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
NROWS = None
# NROWS = 50000

In [3]:
train_identity = pd.read_csv('../input/train_identity.csv', nrows=NROWS)
train_transaction = pd.read_csv('../input/train_transaction.csv', nrows=NROWS)
train = train_transaction.merge(train_identity, how='left', on='TransactionID')

test_identity = pd.read_csv('../input/test_identity.csv', nrows=NROWS)
test_transaction = pd.read_csv('../input/test_transaction.csv', nrows=NROWS)
test = test_transaction.merge(test_identity, how='left', on='TransactionID')

sub = pd.read_csv('../input/sample_submission.csv', nrows=NROWS)

gc.enable()
del train_identity, train_transaction
del test_identity, test_transaction
gc.collect()

print("train.shape:", train.shape)
print("test.shape:", test.shape)
train.head(3)

train.shape: (590540, 434)
test.shape: (506691, 433)


TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  dist2  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0    NaN   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0    NaN   

  P_emaildomain R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0           NaN           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1     gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2   outlook.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   

   C10  C11  C12  C13  C14    D1  D2    D3   D4  D5  D6  D7  D8  D9   D10  \
0  0.0  2.0  0.0  1.0  1.0  14.0 NaN  13.0  NaN NaN NaN NaN NaN NaN  13.0   
1  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   
2  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   

     D11  D12  D13  D14    D15   M1   M2   M3  M4 M5 M6   M7   M8   M9   V1  \
0   13.0  NaN  NaN  NaN    0.0    T    T    T  M2  F  T  NaN  NaN  NaN  1.0   
1    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN  M0  T  T  NaN  NaN  NaN  NaN   
2  315.0  NaN  NaN  NaN  315.0    T    T    T  M0  F  F    F    F    F  1.0   

    V2   V3   V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  0.0  0.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   

   V17  V18  V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  \
0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   

   V32  V33  V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  \
0  0.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   
2  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   

   V47  V48  V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  \
0  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
2  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   

   V62  V63  V64  V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  \
0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   

   V77  V78  V79  V80  V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  \
0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
2  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   

   V92  V93  V94  V95  V96  V97  V98  V99  V100  V101  V102  V103  V104  V105  \
0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0   0.0   1.0   0.0   0.0   0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   V106  V107  V108  V109  V110  V111  V112  V113  V114  V115  V116  V117  \
0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1

In [4]:
target = "isFraud"

In [5]:
train.head(3)

TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  dist2  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0    NaN   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0    NaN   

  P_emaildomain R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0           NaN           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1     gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2   outlook.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   

   C10  C11  C12  C13  C14    D1  D2    D3   D4  D5  D6  D7  D8  D9   D10  \
0  0.0  2.0  0.0  1.0  1.0  14.0 NaN  13.0  NaN NaN NaN NaN NaN NaN  13.0   
1  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   
2  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   

     D11  D12  D13  D14    D15   M1   M2   M3  M4 M5 M6   M7   M8   M9   V1  \
0   13.0  NaN  NaN  NaN    0.0    T    T    T  M2  F  T  NaN  NaN  NaN  1.0   
1    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN  M0  T  T  NaN  NaN  NaN  NaN   
2  315.0  NaN  NaN  NaN  315.0    T    T    T  M0  F  F    F    F    F  1.0   

    V2   V3   V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  0.0  0.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   

   V17  V18  V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  \
0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   

   V32  V33  V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  \
0  0.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   
2  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   

   V47  V48  V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  \
0  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
2  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   

   V62  V63  V64  V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  \
0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   

   V77  V78  V79  V80  V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  \
0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
2  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   

   V92  V93  V94  V95  V96  V97  V98  V99  V100  V101  V102  V103  V104  V105  \
0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0   0.0   1.0   0.0   0.0   0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   V106  V107  V108  V109  V110  V111  V112  V113  V114  V115  V116  V117  \
0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1

In [6]:
test.head(3)

TransactionID  TransactionDT  TransactionAmt ProductCD  card1  card2  \
0        3663549       18403224           31.95         W  10409  111.0   
1        3663550       18403263           49.00         W   4272  111.0   
2        3663551       18403310          171.00         W   4476  574.0   

   card3 card4  card5  card6  addr1  addr2   dist1  dist2 P_emaildomain  \
0  150.0  visa  226.0  debit  170.0   87.0     1.0    NaN     gmail.com   
1  150.0  visa  226.0  debit  299.0   87.0     4.0    NaN       aol.com   
2  150.0  visa  226.0  debit  472.0   87.0  2635.0    NaN   hotmail.com   

  R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  C10  C11  C12  \
0           NaN  6.0  6.0  0.0  0.0  3.0  4.0  0.0  0.0  6.0  0.0  5.0  1.0   
1           NaN  3.0  2.0  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0  1.0  1.0   
2           NaN  2.0  2.0  0.0  0.0  0.0  5.0  0.0  0.0  4.0  0.0  2.0  0.0   

     C13  C14     D1     D2    D3     D4    D5  D6  D7  D8  D9    D10    D11  \
0  115.0  6.0  419.0  419.0  27.0  398.0  27.0 NaN NaN NaN NaN  418.0  203.0   
1   12.0  2.0  149.0  149.0   7.0  634.0   7.0 NaN NaN NaN NaN  231.0  634.0   
2   22.0  2.0  137.0  137.0  10.0   97.0  10.0 NaN NaN NaN NaN  136.0  136.0   

   D12  D13  D14    D15 M1 M2 M3   M4   M5 M6   M7   M8   M9   V1   V2   V3  \
0  NaN  NaN  NaN  409.0  T  T  F  NaN  NaN  F    T    T    T  1.0  1.0  1.0   
1  NaN  NaN  NaN  634.0  T  F  F   M0  NaN  F  NaN  NaN  NaN  1.0  1.0  1.0   
2  NaN  NaN  NaN   97.0  T  T  F   M0    F  F    F    F    F  1.0  1.0  1.0   

    V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  V17  V18  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
1  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   

   V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  V32  V33  \
0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  1.0  1.0  0.0  0.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  V47  V48  \
0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0   
1  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   
2  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   

   V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  V62  V63  \
0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0   

   V64  V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  V77  V78  \
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0   
2  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  1.0  2.0   

   V79  V80  V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  V92  V93  \
0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
1  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  1.0  2.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   

   V94  V95  V96  V97  V98  V99  V100  V101  V102  V103  V104  V105  V106  \
0  0.0  0.0  1.0  0.0  0.0  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1  0.0  0.0  4.0  1.0  0.0  4.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.0  0.0  5.0  0.0  0.0  3.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   

   V107  V108  V109  V110  V111  V112  V113  V114  V115  V116  V117  V118  \
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  

In [7]:
print(train.shape)

(590540, 434)


In [8]:
print(test.shape)

(506691, 433)


In [9]:
test[target] = -1

In [10]:
df = train.append(test)
df.reset_index()

index     C1   C10   C11  C12    C13    C14     C2   C3   C4     C5  \
0             0    1.0   0.0   2.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
1             1    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
2             2    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
3             3    2.0   0.0   1.0  0.0   25.0    1.0    5.0  0.0  0.0    0.0   
4             4    1.0   1.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
5             5    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
6             6    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
7             7    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
8             8    1.0   1.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
9             9    2.0   0.0   1.0  0.0   12.0    2.0    2.0  0.0  0.0    0.0   
10           10    1.0   1.0   2.0  2.0    2.0    1.0    4.0  0.0  1.0    0.0   
11           11    1.0   1.0   1.0  1.0    1.0    1.0    1.0  0.0  1.0    0.0   
12           12    4.0   0.0   2.0  0.0    6.0    3.0    2.0  0.0  0.0    0.0   
13           13    6.0   0.0   5.0  0.0   24.0    6.0    5.0  0.0  0.0    2.0   
14           14    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
15           15    4.0   0.0   3.0  0.0   22.0    3.0    4.0  0.0  0.0    1.0   
16           16    1.0   1.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
17           17    1.0   1.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
18           18    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    1.0   
19           19    2.0   0.0   2.0  0.0   25.0    2.0    5.0  0.0  0.0    0.0   
20           20    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
21           21  127.0   0.0  80.0  0.0  673.0  111.0  120.0  0.0  0.0  168.0   
22           22    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
23           23    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    1.0   
24           24    3.0   0.0   4.0  0.0   10.0    3.0    5.0  0.0  0.0    3.0   
25           25    4.0   0.0   1.0  0.0    5.0    3.0    1.0  0.0  0.0    3.0   
26           26    2.0   0.0   1.0  0.0    4.0    2.0    1.0  0.0  0.0    0.0   
27           27    4.0   0.0   3.0  0.0    4.0    3.0    2.0  0.0  0.0    1.0   
28           28    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
29           29    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
...         ...    ...   ...   ...  ...    ...    ...    ...  ...  ...    ...   
1097201  506661    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
1097202  506662    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    0.0   
1097203  506663    3.0   0.0   1.0  0.0    3.0    3.0    1.0  0.0  0.0    1.0   
1097204  506664    2.0   0.0   1.0  1.0   14.0    2.0    1.0  0.0  0.0    0.0   
1097205  506665    1.0   1.0   1.0  1.0    1.0    1.0    1.0  0.0  1.0    0.0   
1097206  506666   10.0   0.0   1.0  0.0   30.0    6.0    3.0  0.0  0.0    0.0   
1097207  506667    1.0   0.0   2.0  2.0    3.0    1.0    1.0  0.0  0.0    0.0   
1097208  506668    3.0   0.0   1.0  0.0    8.0    3.0    2.0  0.0  0.0    3.0   
1097209  506669    1.0   0.0   1.0  0.0    2.0    1.0    1.0  0.0  0.0    0.0   
1097210  506670    1.0   5.0   1.0  1.0    7.0    1.0    7.0  0.0  2.0    0.0   
1097211  506671    1.0   0.0   1.0  0.0    1.0    1.0    1.0  0.0  0.0    1.0   
1097212  506672    1.0   0.0   1.0  0.0    2.0    1.0    1.0  0.0  0.0    0.0   
1097213  506673    5.0   0.0   3.0  2.0  156.0    4.0    5.0  0.0  0.0    0.0   
1097214  506674    1.0   1.0   1.0  1.0    1.0    1.0    1.0  0.0  1.0    0.0   
1097215  506675    1.0   1.0   1.0  1.0    1.0    1.0    1.0  0.0  1.0    0.0   
1097216  506676    1.0   1.0   1.0  1.0    1.0    1.0    1.0  0.0  1.0    0.0   
1097217  506677    3.0  10.0   3.0  3.0   10.0    3.0   10.0  0.0  3.0    0.0   
1097218  506678    1.0   0.0   1.0  0.0  

In [12]:
df.shape

(1097231, 434)

In [44]:
df['uid'] = df["card1"].apply(lambda x: str(x)) + "_" + df["card2"].apply(lambda x: str(x)) +\
                "_" + df["card3"].apply(lambda x: str(x)) + "_" + df["card4"].apply(lambda x: str(x)) +\
                "_" + df["card5"].apply(lambda x: str(x)) + "_" + df["card6"].apply(lambda x: str(x)) +\
                "_" + df["addr1"].apply(lambda x: str(x)) + "_" + df["addr2"].apply(lambda x: str(x))

In [45]:
df["day"] = df["TransactionDT"] // (24 * 60 * 60)

In [77]:
df[["uid", target, "D10", "day", "TransactionDT"]].head()

uid  isFraud   D10  day  \
0   13926_nan_150.0_discover_142.0_credit_315.0_87.0        0  13.0    1   
1  2755_404.0_150.0_mastercard_102.0_credit_325.0...        0   0.0    1   
2       4663_490.0_150.0_visa_166.0_debit_330.0_87.0        0   0.0    1   
3  18132_567.0_150.0_mastercard_117.0_debit_476.0...        0  84.0    1   
4  4497_514.0_150.0_mastercard_102.0_credit_420.0...        0   NaN    1   

   TransactionDT  
0          86400  
1          86401  
2          86469  
3          86499  
4          86506

In [164]:
df.loc[(df["D15"] == 1) & (df["day"] == 8) & (df["isFraud"] == 0), ["uid", target, "D10", "day", "TransactionDT"]]

uid  isFraud  D10  day  \
27754      11839_490.0_150.0_visa_226.0_debit_123.0_87.0        0  1.0    8   
27959  15219_555.0_131.0_mastercard_166.0_debit_161.0...        0  1.0    8   
27974  2114_396.0_150.0_mastercard_224.0_debit_315.0_...        0  1.0    8   
27994      10776_512.0_150.0_visa_166.0_debit_194.0_87.0        0  0.0    8   
28009      10776_512.0_150.0_visa_166.0_debit_194.0_87.0        0  1.0    8   
28076  5700_122.0_150.0_mastercard_166.0_debit_204.0_...        0  1.0    8   
29015       3681_513.0_150.0_visa_166.0_debit_299.0_87.0        0  1.0    8   
29125      15651_417.0_150.0_visa_226.0_debit_330.0_87.0        0  0.0    8   
29241  10960_567.0_150.0_mastercard_117.0_debit_272.0...        0  1.0    8   
29351       9633_130.0_185.0_visa_138.0_debit_465.0_60.0        0  3.0    8   
29366     12098_555.0_131.0_visa_226.0_credit_430.0_13.0        0  0.0    8   
29671  7919_194.0_150.0_mastercard_202.0_debit_441.0_...        0  1.0    8   
29713      12695_490.0_150.0_visa_226.0_debit_325.0_87.0        0  1.0    8   
29777      13534_105.0_150.0_visa_226.0_debit_110.0_87.0        0  1.0    8   
29782  7919_194.0_150.0_mastercard_202.0_debit_441.0_...        0  1.0    8   
29919      12501_490.0_150.0_visa_226.0_debit_204.0_87.0        0  1.0    8   
29939  7815_161.0_150.0_mastercard_117.0_debit_203.0_...        0  1.0    8   
30203      15885_545.0_185.0_visa_138.0_debit_431.0_60.0        0  1.0    8   
30373  8431_269.0_150.0_mastercard_224.0_debit_433.0_...        0  1.0    8   
30415     12668_170.0_150.0_visa_226.0_credit_325.0_87.0        0  1.0    8   
30494  1618_360.0_150.0_mastercard_166.0_debit_204.0_...        0  1.0    8   
30729      16706_508.0_150.0_visa_226.0_debit_441.0_87.0        0  1.0    8   
30743  3055_474.0_150.0_mastercard_224.0_debit_441.0_...        0  1.0    8   
30846       8320_476.0_150.0_visa_166.0_debit_441.0_87.0        0  0.0    8   
31146       2597_417.0_150.0_visa_195.0_debit_472.0_87.0        0  1.0    8   
31230  11771_543.0_150.0_mastercard_224.0_debit_433.0...        0  1.0    8   
31267     12668_170.0_150.0_visa_226.0_credit_325.0_87.0        0  1.0    8   
31424  10960_567.0_150.0_mastercard_117.0_debit_272.0...        0  0.0    8   

       TransactionDT  
27754         693861  
27959         697132  
27974         697426  
27994         697886  
28009         698172  
28076         699607  
29015         744995  
29125         746762  
29241         748402  
29351         749814  
29366         750008  
29671         753801  
29713         754405  
29777         755135  
29782         755191  
29919         756948  
29939         757159  
30203         760389  
30373         762477  
30415         763035  
30494         764149  
30729         767062  
30743         767172  
30846         768294  
31146         771864  
31230         772780  
31267         773296  
31424         775269

In [165]:
df.loc[(df["D15"] == 1) & (df["day"] == 8) & (df["isFraud"] == 0), "uid"].values

array(['11839_490.0_150.0_visa_226.0_debit_123.0_87.0',
       '15219_555.0_131.0_mastercard_166.0_debit_161.0_60.0',
       '2114_396.0_150.0_mastercard_224.0_debit_315.0_87.0',
       '10776_512.0_150.0_visa_166.0_debit_194.0_87.0',
       '10776_512.0_150.0_visa_166.0_debit_194.0_87.0',
       '5700_122.0_150.0_mastercard_166.0_debit_204.0_87.0',
       '3681_513.0_150.0_visa_166.0_debit_299.0_87.0',
       '15651_417.0_150.0_visa_226.0_debit_330.0_87.0',
       '10960_567.0_150.0_mastercard_117.0_debit_272.0_87.0',
       '9633_130.0_185.0_visa_138.0_debit_465.0_60.0',
       '12098_555.0_131.0_visa_226.0_credit_430.0_13.0',
       '7919_194.0_150.0_mastercard_202.0_debit_441.0_87.0',
       '12695_490.0_150.0_visa_226.0_debit_325.0_87.0',
       '13534_105.0_150.0_visa_226.0_debit_110.0_87.0',
       '7919_194.0_150.0_mastercard_202.0_debit_441.0_87.0',
       '12501_490.0_150.0_visa_226.0_debit_204.0_87.0',
       '7815_161.0_150.0_mastercard_117.0_debit_203.0_87.0',
       '1588

In [175]:
df.loc[(df["uid"] == "10776_512.0_150.0_visa_166.0_debit_194.0_87.0") & (df["day"] == 6), ["uid", target, "D10", "day", "TransactionDT"]]

# day有可能是 7-2,也有可能是7-2-1
# D15 = int(delta秒/3600/24) 不对 3,16 bad case
# D15 = round(delta秒/3600/24)


uid  isFraud  D10  day  \
23716  10776_512.0_150.0_visa_166.0_debit_194.0_87.0        0  0.0    6   

       TransactionDT  
23716         603376

In [177]:
(698172 - 603376)/3600/24

1.097175925925926

In [199]:
feature_list = ["uid", target, "D15", "day", "TransactionDT", "TransactionID"]

In [200]:
df.loc[df["isFraud"] == -1, feature_list].head()

uid  isFraud    D10  day  \
0      10409_111.0_150.0_visa_226.0_debit_170.0_87.0       -1  418.0  213   
1       4272_111.0_150.0_visa_226.0_debit_299.0_87.0       -1  231.0  213   
2       4476_574.0_150.0_visa_226.0_debit_472.0_87.0       -1  136.0  213   
3      10989_360.0_150.0_visa_166.0_debit_205.0_87.0       -1  242.0  213   
4  18018_452.0_150.0_mastercard_117.0_debit_264.0...       -1   22.0  213   

   TransactionDT  TransactionID  
0       18403224        3663549  
1       18403263        3663550  
2       18403310        3663551  
3       18403310        3663552  
4       18403317        3663553

In [269]:
DAY = 214
D15 = 31

In [270]:
df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), feature_list]

uid  isFraud    D10  day  \
2411       2884_490.0_150.0_visa_226.0_debit_204.0_87.0       -1   31.0  214   
2540       9500_321.0_150.0_visa_226.0_debit_204.0_87.0       -1    0.0  214   
2642      7585_553.0_150.0_visa_226.0_credit_143.0_87.0       -1  122.0  214   
3436  18132_567.0_150.0_mastercard_117.0_debit_476.0...       -1  206.0  214   
3453  18132_567.0_150.0_mastercard_117.0_debit_476.0...       -1  206.0  214   
3456  18132_567.0_150.0_mastercard_117.0_debit_476.0...       -1  206.0  214   

      TransactionDT  TransactionID  
2411       18493206        3665960  
2540       18496209        3666089  
2642       18499395        3666191  
3436       18542316        3666985  
3453       18542652        3667002  
3456       18542687        3667005

In [271]:
uid_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), "uid"].values)
print(len(uid_list))
uid_list

6


['2884_490.0_150.0_visa_226.0_debit_204.0_87.0',
 '9500_321.0_150.0_visa_226.0_debit_204.0_87.0',
 '7585_553.0_150.0_visa_226.0_credit_143.0_87.0',
 '18132_567.0_150.0_mastercard_117.0_debit_476.0_87.0',
 '18132_567.0_150.0_mastercard_117.0_debit_476.0_87.0',
 '18132_567.0_150.0_mastercard_117.0_debit_476.0_87.0']

In [274]:
df.loc[(df["uid"] == uid_list[1]) & (df["day"] == DAY - D15), feature_list]

Empty DataFrame
Columns: [uid, isFraud, D10, day, TransactionDT, TransactionID]
Index: []

In [275]:
df.loc[(df["uid"] == uid_list[1]) & (df["day"] == DAY - D15 - 1), feature_list]

uid  isFraud    D10  day  \
587805  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0   17.0  182   
587815  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0   17.0  182   
588512  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  266.0  182   
588525  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  618.0  182   
588598  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  542.0  182   
588676  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
588781  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0   39.0  182   
589056  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        1  118.0  182   
589383  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
589454  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  414.0  182   
589462  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  414.0  182   
589661  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
590060  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
590164  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
590307  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   
590396  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  461.0  182   
590425  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  672.0  182   
590429  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0  672.0  182   
590510  9500_321.0_150.0_visa_226.0_debit_204.0_87.0        0    0.0  182   

        TransactionDT  TransactionID  
587805       15725131        3574805  
587815       15725289        3574815  
588512       15767353        3575512  
588525       15768248        3575525  
588598       15770311        3575598  
588676       15773082        3575676  
588781       15775734        3575781  
589056       15780991        3576056  
589383       15787703        3576383  
589454       15789033        3576454  
589462       15789127        3576462  
589661       15793111        3576661  
590060       15801310        3577060  
590164       15803231        3577164  
590307       15806109        3577307  
590396       15808197        3577396  
590425       15809059        3577425  
590429       15809116        3577429  
590510       15810518        3577510

In [277]:
df.loc[(df["uid"] == uid_list[1]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].mean()

0.05263157894736842

In [238]:
from tqdm import tqdm_notebook

In [306]:
df.loc[df["isFraud"] == -1, "day"].max()

395

In [296]:
fraud_TransactionIDs = []

In [307]:
for DAY in tqdm_notebook(range(213, 395+1)):
    for D15 in range(DAY - 182, DAY - 1):  #212
        uid_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), "uid"].values)
        TransactionID_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), "TransactionID"].values)
        # print(TransactionID_list)
        for i in range(len(uid_list)):
            TransactionID_ = TransactionID_list[i]
            if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list].shape[0] != 0:
                mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].mean()
                sum_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].sum()
            else:
                if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list].shape[0] != 0:
                    mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].mean()
                    sum_  = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].sum()
            if mean_ == 1.0:
                print(TransactionID_, mean_, sum_) 
                fraud_TransactionIDs.append(TransactionID_)

                

3665424 1.0 1
3664786 1.0 1
3665767 1.0 1
3665779 1.0 1
3664532 1.0 2
3664591 1.0 2
3664653 1.0 2
3668073 1.0 1
3668082 1.0 1
3669925 1.0 1
3669682 1.0 1
3670301 1.0 1
3672350 1.0 1
3669005 1.0 1
3672381 1.0 1
3669498 1.0 1
3669046 1.0 2
3669094 1.0 2
3671140 1.0 2
3674192 1.0 1
3674418 1.0 1
3673017 1.0 1
3674342 1.0 1
3674485 1.0 1
3673883 1.0 1
3674657 1.0 1
3672708 1.0 1
3672721 1.0 1
3672695 1.0 1
3676347 1.0 1
3675705 1.0 2
3675431 1.0 1
3676455 1.0 2
3679001 1.0 1
3677768 1.0 1
3679711 1.0 2
3681044 1.0 1
3680605 1.0 1
3680627 1.0 1
3681617 1.0 5
3682183 1.0 5
3682185 1.0 5
3682190 1.0 5
3681000 1.0 5
3681925 1.0 5
3682005 1.0 2
3683209 1.0 1
3684289 1.0 1
3683155 1.0 2
3686365 1.0 1
3687247 1.0 1
3691072 1.0 1
3690835 1.0 1
3690140 1.0 1
3689880 1.0 3
3691005 1.0 3
3689833 1.0 3
3693580 1.0 1
3694428 1.0 1
3696645 1.0 1
3694982 1.0 1
3696115 1.0 1
3694465 1.0 1
3699977 1.0 1
3700513 1.0 1
3701031 1.0 1
3701628 1.0 5
3701639 1.0 5
3701129 1.0 5
3701858 1.0 2
3704387 1.0 1
370190

Exception ignored in: <bound method tqdm.__del__ of  73%|███████▎  | 133/181 [2:02:07<44:04, 55.09s/it]>
Traceback (most recent call last):
  File "/Users/didi/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 883, in __del__
    self.close()
  File "/Users/didi/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 1088, in close
    self._decr_instances(self)
  File "/Users/didi/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 452, in _decr_instances
    cls.monitor.exit()
  File "/Users/didi/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 50, in exit
    self.join()
  File "/Users/didi/anaconda3/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


KeyboardInterrupt: 

In [ ]:
fraud_TransactionIDs

In [224]:
(18418295 - 15474141)/3600/24

34.07585648148148